https://stepik.org/lesson/303592/step/4

[Экспоненциальное скользящее среднее | Exponential Moving Average, EMA](https://allfi.biz/Forex-TechnicalAnalysis-Trend-Indicators-jeksponencialnoe-skolzjashhee-srednee/)

In [ ]:
import torch
import torch.nn as nn

input_size = 3
batch_size = 5
eps = 1e-1

class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum

        # Инициализируем среднее и дисперсию так же, как это делает PyTorch
        self.running_mean = torch.zeros_like(weight)
        self.running_var = torch.ones_like(weight)
        # Хотя и так будет работать:
        # self.running_mean = 0
        # self.running_var = 1

        # По умолчанию модель в режиме обучения
        self.is_training = True

    def __call__(self, input_tensor):
        if self.is_training:
            # Во время обучения вычисляем среднее и смещённую дисперсию по батчу
            mean = torch.mean(input_tensor, dim=0)
            var = torch.var(input_tensor, dim=0, unbiased=False)  # Смещённая дисперсия
            # Для варианта 2:
            # var_unbiased = torch.var(input_tensor, dim=0, unbiased=True)  # Несмещённая дисперсия

            # Обновляем скользящие средние
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var * batch_size / (batch_size - 1)
            # Вариант 2:
            # self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var_unbiased
        else:
            # В режиме предсказания используем накопленные значения
            mean = self.running_mean
            var = self.running_var

        # Нормализация с добавлением eps для стабильности
        std = torch.sqrt(var + self.eps)  # Стандартное отклонение
        normed_tensor = (input_tensor - mean) / std  # Нормализация
        normed_tensor = normed_tensor * self.weight + self.bias  # Масштабирование и сдвиг

        return normed_tensor

    def eval(self):
        # Переключение на режим предсказания
        self.is_training = False

        # Отключаем вычисление градиентов для параметров
        self.weight = self.weight.detach()
        self.bias = self.bias.detach()

# Инициализация стандартной BatchNorm1d для проверки
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

# Создание экземпляра кастомного класса
custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка совпадения результатов
all_correct = True

# Проверка в режиме обучения
for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
        # Проверка совпадения результатов

# Переключение на режим предсказания
batch_norm.eval()
custom_batch_norm1d.eval()

# Проверка в режиме предсказания
for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

# print(all_correct)


True
